# Milestone 1 EDA

The dataset to be used in our project is the Global Food Prices dataset available on the open platform The Humanitarian Data Exchange (HDX) (https://data.humdata.org/dataset/global-wfp-food-prices). This dataset originates from the World Food Programme Price Database and encompasses recorded prices for food items such as maize, rice, beans, fish, and sugar. The price data spans 98 countries and approximately 3,000 markets, and its historical range extends back to 1992 for certain countries, although many countries have started reporting from 2003 onwards. The dataset is updated weekly but primarily comprises monthly data entries, and the data is organized by country due to its extensive volume.

For our initial visualization, we will concentrate on the Japanese market, encompassing data from 2011 to 2020. Following a proof-of-concept review, we intend to expand our visualization for other populated countries. The initial dataset contains approximately 1,180 records of food prices, and each record has 14 variables.


## Import

In [1]:
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [2]:
wfp_jpn = pd.read_csv('../data/raw/wfp_food_prices_jpn.csv', skiprows=[1])
wfp_jpn['date'] = pd.to_datetime(wfp_jpn['date'])

## Pre-Explortion

In [3]:
wfp_jpn.shape

(1177, 14)

In [4]:
wfp_jpn.columns

Index(['date', 'admin1', 'admin2', 'market', 'latitude', 'longitude',
       'category', 'commodity', 'unit', 'priceflag', 'pricetype', 'currency',
       'price', 'usdprice'],
      dtype='object')

In [5]:
wfp_jpn.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,2011-04-15,Oosaka,Oosakasi_Tyuuooku,Osaka,34.69,135.50,cereals and tubers,Rice,5 KG,actual,Retail,JPY,2139.0,25.7370
1,2011-04-15,Oosaka,Oosakasi_Tyuuooku,Osaka,34.69,135.50,cereals and tubers,"Rice (glutinous, unmilled)",KG,actual,Retail,JPY,500.0,6.0161
2,2011-04-15,Tookyoo,Sinzyukuku,Tokyo,35.69,139.69,cereals and tubers,Rice,5 KG,actual,Retail,JPY,2399.0,28.8654
3,2011-05-15,Oosaka,Oosakasi_Tyuuooku,Osaka,34.69,135.50,cereals and tubers,Rice,5 KG,actual,Retail,JPY,2090.0,25.6522
4,2011-05-15,Tookyoo,Sinzyukuku,Tokyo,35.69,139.69,cereals and tubers,Rice,5 KG,actual,Retail,JPY,2375.0,29.1502


In [6]:
wfp_jpn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1177 non-null   datetime64[ns]
 1   admin1     1177 non-null   object        
 2   admin2     1177 non-null   object        
 3   market     1177 non-null   object        
 4   latitude   1177 non-null   float64       
 5   longitude  1177 non-null   float64       
 6   category   1177 non-null   object        
 7   commodity  1177 non-null   object        
 8   unit       1177 non-null   object        
 9   priceflag  1177 non-null   object        
 10  pricetype  1177 non-null   object        
 11  currency   1177 non-null   object        
 12  price      1177 non-null   float64       
 13  usdprice   1177 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(9)
memory usage: 128.9+ KB


In [7]:
for i in list(wfp_jpn.columns):
    print(f"{i:<10}->  {wfp_jpn[i].nunique():<5} unique values")

date      ->  112   unique values
admin1    ->  2     unique values
admin2    ->  2     unique values
market    ->  2     unique values
latitude  ->  2     unique values
longitude ->  2     unique values
category  ->  4     unique values
commodity ->  8     unique values
unit      ->  3     unique values
priceflag ->  1     unique values
pricetype ->  1     unique values
currency  ->  1     unique values
price     ->  473   unique values
usdprice  ->  1140  unique values


## Visualization

In [8]:
wfp_jpn_categorical = list(wfp_jpn.select_dtypes(include = ['object']).columns)
wfp_jpn_float = list(wfp_jpn.select_dtypes(include = ['float64']).columns)

In [9]:
def plot_variables(data: pd.DataFrame, 
                   variables: list, 
                   var_type: str = 'categorical', 
                   ignore_vars: list = None) -> alt.Chart:

    charts = []

    for i, var in enumerate(variables):
        if ignore_vars is not None and var in ignore_vars:
            continue  

        if var_type == 'categorical':
            num_rows = len(data[var].unique())

            chart = alt.Chart(data).mark_bar(stroke=None).encode(
                x=alt.X('count()', title='Count'),
                y=alt.Y(':N'),
                color=alt.Color(':N'),
                row=alt.Row(f'{var}:N')
            ).properties(
                width=280,
                height=280 / num_rows,
                title=f'Bar Plot for {var}',
                spacing=0
            )

        elif var_type == 'continuous':
            hist_chart = alt.Chart(data).mark_bar(opacity=0.7, color='steelblue').encode(
                x=alt.X(f'{var}:Q', bin=alt.Bin(maxbins=50), title=var),
                y=alt.Y('count():Q', stack=None, title='Count'),
            )

            kde_chart = alt.Chart(data).transform_density(
                var,
                as_=[var, 'density'],
            ).mark_line(color='red').encode(
                x=alt.X(f'{var}:Q', title=var),
                y=alt.Y('density:Q', title='Density'),
            )

            chart = alt.layer(hist_chart, kde_chart).resolve_scale(y='independent').properties(
                width=280,
                height=280,
                title=f'Plot for {var}'
            )

        charts.append(chart)

    final_chart = alt.concat(*charts, columns=3).configure_axis(grid=False)

    return final_chart

### Data by Year

In [10]:
wfp_jpn['year'] = wfp_jpn['date'].dt.year
alt.Chart(wfp_jpn).mark_bar(stroke=None).encode(
    x=alt.X('count()', title='Count'),
    y=alt.Y('year:N'),
    color=alt.Color(':N'),
    row=alt.Row(':N')
).properties(
    width=280,
    height=280,
    title=f'Bar Plot for Year',
    spacing=0
)

alt.Chart(...)

### Key Categorical Variables

In [11]:
plot_variables(wfp_jpn, wfp_jpn_categorical, var_type='categorical', ignore_vars=['priceflag', 'pricetype', 'currency'])

alt.ConcatChart(...)

### Key Numerical Variables

In [12]:
plot_variables(wfp_jpn, wfp_jpn_float, var_type='continuous', ignore_vars=['latitude', 'longitude'])

alt.ConcatChart(...)